In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPool2D
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as k

In [ ]:
def create_model():
    # create model
    input_shape = (28,28,1)
    model=Sequential()
    model.add( Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape) )
    model.add( Conv2D(64,kernel_size=(3,3),activation='relu') )
    model.add( MaxPool2D(pool_size=(2,2)) )
    model.add( Conv2D(64,kernel_size=(3,3),activation='relu') )
    model.add( Flatten() )
    model.add( Dense(100,activation='relu') )
    model.add( Dense(10,activation='softmax') )
    # Compile Model
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
             )
    return model
    #model.summary()

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
img_rows, img_cols = 28,28
num_classes = 10
(x_train,y_train),(x_test,y_test) =   mnist.load_data()

if k.image_data_format()=='channels_first':
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
else:
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

input_shape=(img_rows,img_cols,1)
x_train = x_train/255.0
x_test=x_test/255.0
print('x_train shape:',x_train.shape,'\nx_test shape:',x_test.shape)

y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [ ]:
#create model
model=KerasClassifier(build_fn=create_model, verbose=0)
#define grid search parameters
param_grid = {
              'batch_size' : [10],
              'epochs' : [10]
              }
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train,y_train)

In [ ]:
# Results
print('Best: %f using %s' % (grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means,stds,params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#score=model.evaluate(x_test,y_test,verbose=0)
#print('\nTest loss=',score[0],'\nTest Accuracy=',score[1]*100,'%')

In [ ]:
#model.save('Final_simpleCNNmodel.h5')
#print("model is saved")